In [35]:
import pandas as pd
import numpy as np
import math
# import missingno as mn
# import matplotlib.pyplot as plt
# import seaborn as sn
import pickle
from geopy.geocoders import Nominatim

#read in the pubs dataset already processed with distance to their nearest tube line attached
#easiest thing is probably to start adding total travel time to nearest node point of a different line
#again can cover that later, for now just take the closest one
#orrrr just treat them as independant pubs and double the length of your data, it wouldn't add much processing time


# get the user input
def get_coords_for_address(numentries):
    d= {}
    # z = 1
    # while z <= numentries:
    for z in range(numentries):
        result = None
        while result is None:
            d["user_input{0}".format(z)] = input("What's your address? ")
            d["user_input_geocode{0}".format(z)]  = geolocator.geocode(d["user_input{0}".format(z)])
            result = d["user_input_geocode{0}".format(z)]

            try:
                d["user_input_latitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].latitude
                d["user_input_longitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].longitude
            except:
                print("Google can't locate that address! Please try again.")
            z += 1
    return d


# define and get all input data 
def get_pubs_data():
    pubswithdist = pd.read_csv('.\\pubswithdist.csv',index_col=0)
    #read in the already calculated tube network travel time data set
    return pubswithdist

def get_tube_travel():
    data_tubetravel = pd.read_csv('.\\data_tubetravel.csv',index_col=0)
    return data_tubetravel

def get_station_data():
    data_stations = pd.read_csv(".\\stations_csv.sv.csv")
    return data_stations

def get_travel_data():
    #now grab the all important travel times
    data_travel = pd.read_csv(".\\travel_times.csv")
    return data_travel

def get_all_data():
    pubswithdist = get_pubs_data()
    data_tubetravel = get_tube_travel()
    data_sations = get_station_data()
    data_travel = get_travel_data()
    return pubswithdist, data_tubetravel, data_sations, data_travel 

#need to assign every pub a nearest tube
# this should be cached appropriately
#need to define a function for distance between coordinates
#use the haversine formula
#need the angle in radians

#candidate for cython?
def coords_to_distance(x1,x2,y1,y2):
    x1 = x1 * math.pi / 180
    x2 = x2 * math.pi / 180
    y1 = y1 * math.pi / 180
    y2 = y2 * math.pi / 180
    dlat = x2 - x1
    dlon = y2 - y1
    a = (math.pow(math.sin(dlat/2),2)) + math.cos(x1) * math.cos(x2) * (math.pow(math.sin(dlon/2),2))
    c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d = c * 6373
    #this is in km
    return d


# all code after this could be cleaned up and improved
# not convinced by find second station at the moment
def find_second_station(previous_station):
    alreadycheckedlines= []
    for line in data_travel[data_travel['station1'] == previous_station].iterrows():
        #print(line)
        alreadycheckedlines.append(line[1]['line'])
        datatravel_temp = data_travel[~data_travel['line'].isin(alreadycheckedlines)]
        data_filtered = data_stations[data_stations['id'].isin(datatravel_temp['station1'])]
    return data_filtered


geolocator = Nominatim(user_agent= "GoogleV3")
numentries = int(input("How many addresses?"))
get_coords_for_address(numentries)


How many addresses?1
What's your address? london


In [36]:
d = get_coords_for_address(1)

What's your address? london


In [37]:
d

{'user_input0': 'london',
 'user_input_geocode0': Location(London, Greater London, England, SW1A 2DX, United Kingdom, (51.5073219, -0.1276474, 0.0)),
 'user_input_latitude0': 51.5073219,
 'user_input_longitude0': -0.1276474}

In [40]:
d["user_input_latitude{0}".format(0)]

51.5073219

In [22]:
data_stations = get_station_data()
data_stations.head()

,id,latitude,longitude,name,display_name,zone,total_lines,rail
0,1,51.5028,-0.2801,Acton Town,Acton<br />Town,3.0,2,0
1,2,51.5143,-0.0755,Aldgate,NaN,1.0,2,0
2,3,51.5154,-0.0726,Aldgate East,Aldgate<br />East,1.0,2,0
3,4,51.5107,-0.0130,All Saints,All<br />Saints,2.0,1,0
4,5,51.5407,-0.2997,Alperton,NaN,4.0,1,0


In [44]:
data_stations[data_stations['id'] == 49]

,id,latitude,longitude,name,display_name,zone,total_lines,rail
48,49,51.508,-0.1247,Charing Cross,Charing<br />Cross,1.0,2,1


In [23]:
def find_nearest_station(input_lat, input_long, user_num, data_stations):
    min_distance = 1000
    closest_tube_id = []
    closest_tube = []
    for tube in data_stations.itertuples():
        #get distance between input coords and all tubes
        current_dist = coords_to_distance(getattr(tube,'latitude'),input_lat,getattr(tube,'longitude'),input_long)
        if current_dist < min_distance:
            min_distance = current_dist
            closest_tube_id = getattr(tube,'id')
    return closest_tube_id        

In [43]:
# %%timeit
find_nearest_station(d["user_input_latitude{0}".format(0)],d["user_input_longitude{0}".format(0)],1,data_stations)

49

In [98]:
for tube in data_stations.itertuples():
    getattr(tube,'id')

In [46]:
pubswithdist, data_tubetravel, data_sations, data_travel  = get_all_data()

In [ ]:
test_dict

In [89]:
data_tubetravel[data_tubetravel['id'] == int(49)]['travel_times']

48    {1: {'time': 27, 'path': [{'station_id': 196, ...
Name: travel_times, dtype: object

In [104]:
data_tubetravel[data_tubetravel['id'] == int(49)]

,destinations,id,latitude,longitude,name,rail,total_lines,travel_times,zone
48,"[{'station_id': 87, 'name': 'Embankment', 'lin...",49,51.508,-0.1247,Charing Cross,1,2,"{1: {'time': 27, 'path': [{'station_id': 196, ...",1.0


In [82]:
from ast import literal_eval
test_dict = literal_eval(data_tubetravel[data_tubetravel['id'] == int(49)]['travel_times'][48])

In [85]:
test_dict

{1: {'time': 27,
  'path': [{'station_id': 196, 'line_id': 1},
   {'station_id': 107, 'line_id': 10},
   {'station_id': 133, 'line_id': 10},
   {'station_id': 146, 'line_id': 10},
   {'station_id': 235, 'line_id': 10},
   {'station_id': 99, 'line_id': 10},
   {'station_id': 74, 'line_id': 10},
   {'station_id': 17, 'line_id': 10},
   {'station_id': 110, 'line_id': 10},
   {'station_id': 264, 'line_id': 10},
   {'station_id': 1, 'line_id': 10}]},
 2: {'time': 19,
  'path': [{'station_id': 87, 'line_id': 1},
   {'station_id': 254, 'line_id': 3},
   {'station_id': 25, 'line_id': 3},
   {'station_id': 161, 'line_id': 3},
   {'station_id': 44, 'line_id': 3},
   {'station_id': 166, 'line_id': 3},
   {'station_id': 262, 'line_id': 3},
   {'station_id': 2, 'line_id': 3}]},
 3: {'time': 21,
  'path': [{'station_id': 87, 'line_id': 1},
   {'station_id': 254, 'line_id': 3},
   {'station_id': 25, 'line_id': 3},
   {'station_id': 161, 'line_id': 3},
   {'station_id': 44, 'line_id': 3},
   {'station

In [ ]:
# new strategy, instead of working out the nearest station why not take precomputed distance from all stations to all stations
# then compute travel time to all stations (which you already do)
# and just compare total time of all vs all 

In [95]:

# could this not be precomputed?
def nearest_station_to_rest(closest_tube_id):
    person_travel = []
    tube_id = []
    tube_results = []
    
    from ast import literal_eval
    test_dict = literal_eval(data_tubetravel[data_tubetravel['id'] == closest_tube_id]['travel_times'][closest_tube_id - 1])
    
    for destination in test_dict:
        person_travel.append(test_dict[destination]['time'])
        tube_id.append(destination)
    return person_travel, tube_id


In [105]:
# %%timeit
# around 20 seconds to compute distance from every station to every other station
test_person_travel = []
test_tube_id = []

for tube in data_stations.itertuples():
    test_val = getattr(tube,'id')
    person_travel, tube_id = nearest_station_to_rest(test_val)
    test_tube_id.append(tube_id)
    test_person_travel.append(person_travel)

In [109]:
test_person_travel = pd.DataFrame(test_person_travel)

In [122]:
test_person_travel.to_csv(r'./station_to_station_time.csv', index=False)

In [117]:
data_stations[data_stations['id'].isin([145, 7,188,89])]

,id,latitude,longitude,name,display_name,zone,total_lines,rail
6,7,51.5322,-0.1058,Angel,NaN,1.0,1,0
88,89,51.5282,-0.1337,Euston,NaN,1.0,2,1
144,145,51.5308,-0.1238,King's Cross St. Pancras,King's Cross<br />St. Pancras,1.0,6,1
187,188,51.5263,-0.0873,Old Street,Old<br />Street,1.0,1,1


In [119]:
# can just look up travel time now right?
test_person_travel

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,301
0,0,51,53,53,14,62,37,42,53,43,...,31,51,20,52,41,32,29,65,54,48
1,51,0,10,22,61,82,12,26,37,23,...,65,12,44,48,56,63,60,26,38,32
2,53,10,0,20,63,84,14,28,39,25,...,67,2,46,50,58,65,62,26,40,34
3,53,22,20,0,63,84,21,34,49,35,...,67,14,46,50,58,65,62,23,46,44
4,10,61,63,63,0,48,52,61,72,58,...,41,61,30,38,51,42,39,75,73,67
5,58,82,84,84,48,0,62,71,82,68,...,89,82,73,48,76,91,88,96,83,77
6,43,12,18,21,56,66,0,13,28,14,...,54,19,33,32,36,52,49,34,25,23
7,45,25,31,34,55,75,13,0,37,23,...,55,32,34,41,45,57,54,47,12,32
8,56,40,46,49,66,86,28,37,0,14,...,74,47,53,52,56,68,65,62,49,5
9,49,26,32,35,62,72,14,23,14,0,...,60,33,39,38,42,57,54,48,35,9


In [ ]:
# could I also get all the travel times from every pub to every tube?

In [120]:
pubswithdist

,index,0_x,1,2,3,4,5,6,7,8,latitude,longitude,0_y,0,traveltime_totube,traveltime_totube_pub
0,1875,17820,The Three Jolly Wheelers,"""735 Chigwell Road, Woodford Green, Chigwell, ...",IG8 8AS,542653,192011.0,51.608869,0.058699,Redbridge,51.608869,0.058699,214,1.372238,1.633617e+01,1.633617e+01
1,8950,85383,Aria Bar,"""31 Longbridge Road, Barking""",IG11 8TN,544481,184425.0,51.54024,0.081962,Barking and Dagenham,51.540240,0.081962,15,0.097452,1.160141e+00,1.160141e+00
2,8951,85412,Barking Arms Ltd,"""25-27 Station Parade, Barking""",IG11 8TU,544404,184344.0,51.539531,0.080819,Barking and Dagenham,51.539531,0.080819,15,0.014687,1.748431e-01,1.748431e-01
3,8952,85420,Barking football Club,"""Barking Football Club, Lodge Avenue, Dagenham""",RM8 2JR,546614,185058.0,51.545377,0.11296,Barking and Dagenham,51.545377,0.112960,268,1.106634,1.317422e+01,1.317422e+01
4,8953,85423,Barking Indoor Bowls Club Ltd,"""Indoor Bowls Hall, Barking Prk, Longbridge Ro...",IG11 8TA,544692,184653.0,51.542225,0.085096,Barking and Dagenham,51.542225,0.085096,15,0.406872,4.843717e+00,4.843717e+00
5,8954,85429,Barking Rugby Union Football Club,"""Gale Street, Dagenham""",RM9 4TX,547647,183924.0,51.53492,0.127357,Barking and Dagenham,51.534920,0.127357,21,0.598926,7.130071e+00,7.130071e+00
6,8955,85436,Barking United Services Club,"""79 Blake Avenue, Barking""",IG11 9SF,545650,183656.0,51.53303,0.098489,Barking and Dagenham,51.533030,0.098489,268,0.640896,7.629717e+00,7.629717e+00
7,8956,85440,Beacon Tree,"""Beacon Tree, 945 Green Lane, Dagenham""",RM8 1DJ,548489,186902.0,51.561457,0.140758,Barking and Dagenham,51.561457,0.140758,67,2.238250,2.664583e+01,2.664583e+01
8,8957,85519,Chadwell Heath Cricket Club,"""St Chads Park, West Road, Chadwell Heath""",RM6 6YA,547980,188485.0,51.575822,0.134086,Barking and Dagenham,51.575822,0.134086,173,3.054550,3.636369e+01,3.636369e+01
9,8958,85589,Dagenham Bowling,"""Dagenham Leisure Park, Cook Road, Dagenham""",RM9 6UQ,548237,183680.0,51.532573,0.135769,Barking and Dagenham,51.532573,0.135769,21,1.052035,1.252423e+01,1.252423e+01


In [90]:
#need this to be self contained, and output an identifiable dataset ready for further processing
def distance_to_pubs(input_lat, input_long, user_num, data_stations):
    closest_tube_id = find_nearest_station(input_lat,input_long,user_num,data_stations)   
    
    
    #this loop returns the closest tube station to the input coords
    #then the below works out the travel time to all the things from that
    #data tubetravel contains travel time from each tube station to every other tube station
    #so just filter to the nearest tube station and check time to the rest of the network
    #need to check each line as well
    

    person_travel = []
    tube_id = []
    tube_results = []
    
    from ast import literal_eval
    test_dict = literal_eval(data_tubetravel[data_tubetravel['id'] == closest_tube_id]['travel_times'][closest_tube_id - 1])
    
    for destination in test_dict:
        person_travel.append(test_dict[destination]['time'])
        tube_id.append(destination)
    
#     for line in data_tubetravel[data_tubetravel['id'] == closest_tube_id]['travel_times']:
#         while x < len(data_stations):
#             person_travel.append(line[x]['time']) 
#             tube_id.append(x)
#             x += 1

    #the above returns the travel time to all stations from the nearest station
    user_num = str(user_num)    
    person_travel = pd.DataFrame(person_travel)
    tube_id = pd.DataFrame(tube_id)
    individ_tube = pd.merge(tube_id,person_travel,left_index = True, right_index = True)
    individ_tube.columns = ['tube_id','travel_time_to_tube'+user_num]
    
    
    full_travel = pd.merge(pubs_with_dist_filt,individ_tube, how = 'left',left_on = '0_y',right_on = 'tube_id')
    min_distance_time = min_distance / 0.084
    full_travel['total time' + user_num] = full_travel['traveltime_totube_pub'] + full_travel['travel_time_to_tube'+str(user_num)] + min_distance_time
    return full_travel, closest_tube_id



In [ ]:
#could be worth filtering pubs with dist to only a couple of columns so that it's neater?
#could be worth filtering pubs with dist to only a couple of columns so that it's neater?
pubs_with_dist_filt = pubswithdist[['0_y','index','traveltime_totube_pub']]
numpeeps = 2
i = 1
while i <= numpeeps:
    full_travel_i_1, i_closest_tube_id = distance_to_pubs(d["user_input_latitude{0}".format(i)],d["user_input_longitude{0}".format(i)],i,data_stations)
    full_travel_i_2, i_second_closest_tube = distance_to_pubs(d["user_input_latitude{0}".format(i)],d["user_input_longitude{0}".format(i)],i,find_second_station(i_closest_tube_id))
    full_travel_i = pd.merge(full_travel_i_1,full_travel_i_2, left_index=True,right_index=True)
    #full_travel_i['True_shortest' + str(i)] = min(full_travel_i['total time' + str(i) + '_x'], full_travel_i['total time' + str(i) + '_y'])
    full_travel_i['true_shortest' + str(i)] = full_travel_i[['total time' + str(i) + '_x', 'total time' + str(i) + '_y']].min(axis=1)
    full_travel_i_final = full_travel_i[['true_shortest' + str(i)]]
    if i > 1:
        full_current = pd.merge(full_travel_i_final,full_current,left_index=True,right_index=True)
    else:
        full_current = full_travel_i_final
    i += 1

In [ ]:
cols_to_use = ['true_shortest' + str(p+1) for p in range(numpeeps)]
#full_current['Total'] = full_current.loc[:, cols_to_use].sum(axis=1)
full_current['Total'] = full_current[cols_to_use].sum(axis =1 )
full_current = pd.merge(full_current,pubswithdist,left_index= True,right_index=True)
full_current = full_current[full_current['Total'] != 0]


finalfinal = full_current.sort_values('Total')
finalfinal_top = finalfinal.head(10)
finalfinal_top = finalfinal_top[['Total','2','8']]
print(finalfinal_top)